###################################################
#                                                 #
#              연      습      중                   #
#                                                 #
###################################################

```
faiss 까지 들어가 있는 데이터를 기반으로
1. query 를 분석
2. 현재까지는 (상품명,브랜드명) && (품목+카테고리+키워드) && (가격범위||가격대) 를 필터하도록 구성
3. 각 조건에 따라 가중치를 별도로 두는 방식으로 진행  
    --> eg. LG전자 가 query 에 있을 경우  
        --> 상품명에 포함되어 있으면 가점 3.0  
                브랜드명에 포함되어 있으면 가점 4.0  
4. 각 가중치에 따라 대용량 처리 진행 (faiss 에 몇 천만 rows 이상 있다는 가정 하) --> hybrid_search
5. 4번에서 나온 결과로 LLM 을 추가 처리하여 재 정렬 시도 --> rerank_with_llm

** 현재까지 지원되는 조건 (QueryConditions 를 참고)
 - 브랜드
 - 품목
 - 카드 할인
 - 가격 (금액대, 가격 구간)
 - 평점, 별점
 - 에너지 소비 효율 등급
 - 모델 출시일 (년, 월 만 활용)
 - 리뷰 개수 (정렬용)
 - 판매량(한달) (정렬용)
 - 단위판매량 (정렬용)

비고
1. 카드할인의 경우 '할인되는, 할인하는' 으로 정리 --> 추후 사전 정의 필요
2. 브랜드의 경우 여러가지 입력될 수 있는 가능성을 두고 사전 정의 --> 전체 상품 데이터 확인 후 사전 데이터 보강 필요
3. 품목+카테고리의 경우에도 마찬가지로 입력될 수 있는 가능성을 두고 사전 정의 --> 전체 상품 데이터 확인 후 사전 데이터 보강 필요
4. 금액구간 및 금액대의 경우는 한국 원 단위로 쓸 수 있는 경우를 대부분 고려함
--> 단, 현재는 1만원 미만으로는 고려되지 않음
--> 1~3번 조건과는 달리 '가중치'로 반영되는 조건이 아니라, 구간에 맞으면 통과되는 filter 조건으로 적용
4. 가중치를 변경하면서 어떤 내용이 매칭되냐에 따라 결과를 달리 볼 수 있음
--> calculate_XXXXXX_score 부분에서 각자의 가중치 관리 가능
--> process_batch 부분에서 라이브러리가 계산한 '벡터 코사인 유사도 점수' 와 가중치에 따라 사용자가 계산한 '점수' 에 대한 2번째 가중치를 설정 가능 (합이 1이 되도록)
5. 대량 수행 (현재 약 100,000건 정도)을 병렬로 처리할 수 있는 방식으로 구현 (process_batch + hybrid_search)
6. hybrid_search 의 결과 개수와 rerank_with_llm 의 결과 개수가 차이나지 않도록 보완
```

In [148]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Tuple, Optional, Union
import time
import re
from dataclasses import dataclass
from datetime import datetime, timedelta

# 임베딩 모델과 벡터스토어 초기화
embedding_model = OpenAIEmbeddings()
persist_directory = "../.db/faiss"
vectorstore = FAISS.load_local(persist_directory, embedding_model, allow_dangerous_deserialization=True)

In [149]:
@dataclass
class QueryConditions:
    brands: List[str]
    items: List[str]
    card_discounts: List[str]
    price_conditions: List[Tuple[str, int]]
    price_band_conditions: List[Tuple[str, int, int]]
    gdas_scr_sum: Optional[Tuple[float, str]]
    energy_grade: Optional[Tuple[str, str]]
    mdl_lnch_dt: Optional[Tuple[str, int, int]]
    # 정렬 조건은 'asc' 또는 'desc' 중 하나
    sort_gdas_cnt: Optional[str]
    sort_sale_qty: Optional[str]
    sort_sales_unit: Optional[str]

In [150]:
def analyze_brands(query: str) -> List[str]:
    """브랜드 정보 추출 함수"""
    brand_mapping = {
        'LG': 'LG전자',
        'Lg': 'LG전자',
        'lg': 'LG전자',
        '엘지': 'LG전자',
        '엘쥐': 'LG전자',
        '엘지전자': 'LG전자',
        'Samsung': '삼성전자',
        'samsung': '삼성전자',
        'SAMSUNG': '삼성전자',
        '삼성': '삼성전자',
        'Apple': '애플',
        'apple': '애플',
        'APPLE': '애플',
        'Sony': '소니',
        'sony': '소니',
        'SONY': '소니',
        'Prism': '프리즘코리아',
        'prism': '프리즘코리아',
        'PRISM': '프리즘코리아',
        '프리즘': '프리즘코리아',
        'Lucoms': '루컴즈',
        'lucoms': '루컴즈',
        'LUCOMS': '루컴즈',
        'Nex': '넥스',
        'nex': '넥스',
        'NEX': '넥스',
        'TCL': 'TCL',
        'Xiaomi': '샤오미',
        'xiaomi': '샤오미',
        'XIAOMI': '샤오미',
        'Philips': '필립스',
        'philips': '필립스',
        'PHILIPS': '필립스'
    }
    
    brand_pattern = r'(LG(?:전자)?|삼성(?:전자)?|애플|소니|프리즘(?:코리아)?|루컴즈|넥스|TCL|샤오미|필립스)'
    brands_found = re.findall(brand_pattern, query, re.IGNORECASE)
    
    brands = []
    for brand in brands_found:
        normalized_brand = brand.upper()
        for key, value in brand_mapping.items():
            if key.upper() in normalized_brand:
                if value not in brands:
                    brands.append(value)
                break
    
    return brands

In [151]:
def analyze_items(query: str) -> List[str]:
    """품목 정보 추출 함수"""
    item_pattern = r'\b(TV|냉장고|세탁기|건조기|에어컨|공기청정기)\b'
    matches = re.findall(item_pattern, query, re.IGNORECASE)
    
    item_mapping = {
        'tv': 'TV',
        'Tv': 'TV',
        'tV': 'TV'
    }
    
    return [item_mapping.get(item, item) for item in matches]

In [152]:
def analyze_card_discounts(query: str) -> List[str]:
    """카드할인 정보 추출 함수"""
    discount_pattern = r'(\w+카드)\s*(할인하는|할인되는)'
    return [card for card, _ in re.findall(discount_pattern, query)]

In [153]:
def calculate_price_range(amount: int) -> tuple[int, int]:
    """가격대 범위 계산 함수"""
    if amount == 0:
        return (0, 0)
    
    str_amount = str(amount)
    first_non_zero = 0
    for i, digit in enumerate(str_amount):
        if digit != '0':
            first_non_zero = i
            break
    
    remaining_digits = len(str_amount) - first_non_zero - 1
    start_amount = amount
    first_part = int(str_amount[first_non_zero]) + 1
    end_amount = first_part * (10 ** remaining_digits)
    end_amount = int(str_amount[:first_non_zero] + str(end_amount))
    
    return (start_amount, end_amount)

def convert_korean_number(number: str, unit: str) -> tuple[int, int]:
    """한국어 숫자 단위 변환 함수"""
    multiplier = 1
    if '십' in number:
        multiplier = 10
        number = number.replace('십', '')
    elif '백' in number:
        multiplier = 100
        number = number.replace('백', '')
    elif '천' in number:
        multiplier = 1000
        number = number.replace('천', '')
    
    unit_multiplier = {
        '천': 1000,
        '만': 10000,
        '억': 100000000
    }
    
    base_amount = int(number) * multiplier * unit_multiplier[unit]
    return calculate_price_range(base_amount)

In [154]:
def analyze_price_conditions(query: str) -> Tuple[List[Tuple[str, int]], List[Tuple[str, int, int]]]:
    """가격 조건 분석 함수"""
    price_conditions = []
    price_band_conditions = []
    
    # 구간대 패턴
    price_range_pattern = r'((?:\d+)?(?:십|백|천)?(?:\d+)?)\s*(천|만|억)원\s*(이상|이하|초과|미만)'
    price_ranges = re.findall(price_range_pattern, query)
    
    for amount, unit, condition in price_ranges:
        base_amount = convert_korean_number(amount, unit)[0]  # 시작 금액만 사용
        
        if condition == '이상':
            price_conditions.append(('>=', base_amount))
        elif condition == '이하':
            price_conditions.append(('<=', base_amount))
        elif condition == '초과':
            price_conditions.append(('>', base_amount))
        elif condition == '미만':
            price_conditions.append(('<', base_amount))
    
    # 금액대 패턴
    price_range_band_pattern = r'((?:\d+)?(?:십|백|천)?(?:\d+)?)\s*(천|만|억)원대'
    price_bands = re.findall(price_range_band_pattern, query)
    
    for amount, unit in price_bands:
        start_amount, end_amount = convert_korean_number(amount, unit)
        price_band_conditions.append(('band', start_amount, end_amount))
    
    return price_conditions, price_band_conditions

In [155]:
def analyze_gdas_scr_sum(query: str) -> Optional[Tuple[float, str]]:
    """
    가격 조건 분석 함수
    "이상" 및 "초과" 만 대응, '넘는' 등은 추후 고도화 필요
    """
    pattern = r"(평점|별점)\s*([\d.]+)\s*(점)?\s*(이상|초과)"
    match = re.search(pattern, query)

    if match:
        value = float(match.group(2))
        operator = match.group(4)
        if operator == "이상":
            return (value, ">=")
        elif operator == "초과":
            return (value, ">")
    
    return None

In [156]:
def analyze_energy_grade(query: str) -> Optional[Tuple[str, str]]:
    """
    에너지등급 조건 분석 함수
    """
    energy_keywords = ['등급', '에너지등급', '에너지효율', '에너지소비효율', '에너지소비효율등급']
    has_energy_keyword = any(keyword in query for keyword in energy_keywords)
    
    if not has_energy_keyword:
        return None
    
    grade_pattern = r'(\d+)\s*등급(?:\s*(이상))?'
    match = re.search(grade_pattern, query)
    
    if match:
        grade_num = int(match.group(1))
        if 1 <= grade_num <= 5:
            grade = f"{grade_num}등급"
            condition = 'above' if match.group(2) else 'equal'
            return (grade, condition)
    
    return None

In [157]:
def analyze_mdl_lnch_dt(query: str) -> Optional[Union[Tuple[str, int, int], Tuple[str, int, int, int, int]]]:
    """
    제조년월 조건 분석 함수 (YYYYMMDD 중 DD는 무시)
    
    반환 타입:
    - 정확 일치: ('exact', 년도, 월)
    - 범위: ('range', 시작년도, 시작월, 끝년도, 끝월)
    - 기간: ('period', 년도, 월, 개월수)  # 예: ('period', 2025, 3, 2) = 2025년 3월부터 2개월
    
    지원 패턴:
    - 정확: "2025년 3월 생산", "올해 이번달 모델"
    - 범위: "2025년 3월 이후", "2024년 6월~2025년 2월", "작년 3월부터 올해 2월까지"
    - 기간: "2개월 이내", "3개월 전부터", "최근 6개월"
    """
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month

    production_keywords = ['생산', '제조', '모델', '출시', '연식', '년식']
    if not any(keyword in query for keyword in production_keywords):
        return None

    # 1. 기간 패턴
    period_patterns = [
        (r'(\d+)개월\s*이내', 1),  # 2개월 이내
        (r'최근\s*(\d+)개월', 1),  # 최근 3개월
        (r'(\d+)개월\s*전부터', -1),  # 3개월 전부터
        (r'지난\s*(\d+)개월', 1),  # 지난 2개월
    ]
    
    for pattern, direction in period_patterns:
        m = re.search(pattern, query)
        if m:
            months = int(m.group(1))
            if direction == 1:  # 현재부터 과거로
                end_date = current_date
                start_date = current_date - timedelta(days=months*30)
            else:  # 과거부터 현재로
                start_date = current_date - timedelta(days=months*30)
                end_date = current_date
            
            return ('period', start_date.year, start_date.month, end_date.year, end_date.month)

    # 2. 범위 패턴
    range_patterns = [
        (r'(\d{4})년\s*(\d{1,2})월\s*이후', 'after'),
        (r'(\d{4})년\s*(\d{1,2})월\s*부터', 'after'),
        (r'(\d{4})년\s*(\d{1,2})월\s*이전', 'before'),
        (r'(\d{4})년\s*(\d{1,2})월\s*까지', 'before'),
    ]
    
    for pattern, range_type in range_patterns:
        m = re.search(pattern, query)
        if m:
            year = int(m.group(1))
            month = int(m.group(2))
            if range_type == 'after':
                return ('range', year, month, current_year, current_month)
            else:
                return ('range', 1900, 1, year, month)

    # 3. 구간 패턴
    interval_patterns = [
        (r'(\d{4})년\s*(\d{1,2})월\s*[~-]\s*(\d{4})년\s*(\d{1,2})월', 'exact_range'),
        (r'(\d{4})년\s*(\d{1,2})월\s*부터\s*(\d{4})년\s*(\d{1,2})월\s*까지', 'exact_range'),
    ]
    
    for pattern, range_type in interval_patterns:
        m = re.search(pattern, query)
        if m:
            start_year = int(m.group(1))
            start_month = int(m.group(2))
            end_year = int(m.group(3))
            end_month = int(m.group(4))
            return ('range', start_year, start_month, end_year, end_month)

    # 4. 상대적 구간 패턴
    relative_patterns = [
        (r'작년\s*(\d{1,2})월\s*부터\s*올해\s*(\d{1,2})월\s*까지', (current_year-1, current_year)),
        (r'지난해\s*(\d{1,2})월\s*부터\s*금년\s*(\d{1,2})월\s*까지', (current_year-1, current_year)),
    ]
    
    for pattern, (start_year, end_year) in relative_patterns:
        m = re.search(pattern, query)
        if m:
            start_month = int(m.group(1))
            end_month = int(m.group(2))
            return ('range', start_year, start_month, end_year, end_month)

    # 5. 정확 일치 패턴
    year_patterns = [
        (r'올해', current_year),
        (r'금년', current_year),
        (r'작년', current_year - 1),
        (r'지난해', current_year - 1),
        (r'([12][0-9]{3})년', None),
    ]
    
    month_patterns = [
        (r'이번달', current_month),
        (r'금월', current_month),
        (r'지난달', current_month - 1 if current_month > 1 else 12),
        (r'([1-9]|1[0-2])월', None),
    ]

    target_year = None
    target_month = None

    for pattern, default_year in year_patterns:
        m = re.search(pattern, query)
        if m:
            if default_year is not None:
                target_year = default_year
            else:
                target_year = int(m.group(1))
            break

    for pattern, default_month in month_patterns:
        m = re.search(pattern, query)
        if m:
            if default_month is not None:
                target_month = default_month
            else:
                target_month = int(m.group(1))
            break

    if target_year is not None and target_month is None:
        target_month = current_month
    if target_month is not None and target_year is None:
        target_year = current_year

    if target_year is not None and target_month is not None:
        if 1 <= target_month <= 12:
            return ('exact', target_year, target_month)
    
    return None

In [158]:
def analyze_gdas_cnt(query: str) -> Optional[str]:
    """리뷰개수 정렬 조건 분석 함수"""
    gdas_keywords = ['리뷰개수', '리뷰 수', '리뷰수', '평점개수', '평점 수', '리뷰', '평점', '별점']
    has_gdas_keyword = any(keyword in query for keyword in gdas_keywords)
    
    if not has_gdas_keyword:
        return None
    
    if any(word in query for word in ['많은 순서로', '많은순', '많은 순', '높은 순서로', '높은순']):
        return 'desc'
    elif any(word in query for word in ['적은 순서로', '적은순', '적은 순', '낮은 순서로', '낮은순']):
        return 'asc'
    
    return None

In [159]:
def analyze_sale_qty(query: str) -> Optional[str]:
    """판매량(한달) 정렬 조건 분석 함수"""
    # 많이 팔린 순, 적게 팔린 순 등 대응 추가 필요
    sale_qty_keywords = ['판매량', '판매 수량', '월 판매량', '한달 판매량', '월별 판매량']
    has_sale_qty_keyword = any(keyword in query for keyword in sale_qty_keywords)
    
    if not has_sale_qty_keyword:
        return None
    
    if any(word in query for word in ['많은 순서로', '많은순', '많은 순', '높은 순서로', '높은순']):
        return 'desc'
    elif any(word in query for word in ['적은 순서로', '적은순', '적은 순', '낮은 순서로', '낮은순']):
        return 'asc'
    
    return None

In [160]:
def analyze_sales_unit(query: str) -> Optional[str]:
    """단위판매량 정렬 조건 분석 함수"""
    sales_unit_keywords = ['단위판매량', '단위 판매량', '개당 판매량', '단가별 판매량']
    has_sales_unit_keyword = any(keyword in query for keyword in sales_unit_keywords)
    
    if not has_sales_unit_keyword:
        return None
    
    if any(word in query for word in ['많은 순서로', '많은순', '많은 순', '높은 순서로', '높은순']):
        return 'desc'
    elif any(word in query for word in ['적은 순서로', '적은순', '적은 순', '낮은 순서로', '낮은순']):
        return 'asc'
    
    return None

In [161]:
def analyze_query(query: str) -> QueryConditions:
    """쿼리 분석 통합 함수"""
    # 각 조건 분석
    brands = analyze_brands(query)
    items = analyze_items(query)
    card_discounts = analyze_card_discounts(query)
    price_conditions, price_band_conditions = analyze_price_conditions(query)
    gdas_scr_sum_cond = analyze_gdas_scr_sum(query)
    energy_grade_cond = analyze_energy_grade(query)
    mdl_lnch_dt_cond = analyze_mdl_lnch_dt(query)
    sort_gdas_cnt = analyze_gdas_cnt(query)
    sort_sale_qty = analyze_sale_qty(query)
    sort_sales_unit = analyze_sales_unit(query)
    
    # 결과 출력
    print(f"\n✅ 브랜드 :: {brands}")
    print(f"\n✅ 아이템 :: {items}")
    print(f"\n✅ 카드할인 :: {card_discounts}")
    print(f"\n✅ 가격 조건 :: {price_conditions}")
    print(f"\n✅ 가격대 :: {price_band_conditions}")
    print(f"\n✅ 평점 :: {gdas_scr_sum_cond}")
    print(f"\n✅ 에너지등급 :: {energy_grade_cond}")
    print(f"\n✅ 제조년월 :: {mdl_lnch_dt_cond}")
    print(f"\n✅ 리뷰개수 정렬 :: {sort_gdas_cnt}")
    print(f"\n✅ 판매량 정렬 :: {sort_sale_qty}")
    print(f"\n✅ 단위판매량 정렬 :: {sort_sales_unit}")
    
    return QueryConditions(
        brands=brands,
        items=items,
        card_discounts=card_discounts,
        price_conditions=price_conditions,
        price_band_conditions=price_band_conditions,
        gdas_scr_sum=gdas_scr_sum_cond,
        energy_grade=energy_grade_cond,
        mdl_lnch_dt=mdl_lnch_dt_cond,
        sort_gdas_cnt=sort_gdas_cnt,
        sort_sale_qty=sort_sale_qty,
        sort_sales_unit=sort_sales_unit
    )

In [162]:
def get_effective_price(metadata: dict) -> int:
    """메타데이터에서 실제 적용될 가격을 반환하는 함수"""
    max_benefit_price = int(metadata.get('MAX_BENEFIT_PRICE', 0))
    dscnt_sale_prc = int(metadata.get('DSCNT_SALE_PRC', 0))
    sale_prc = int(metadata.get('SALE_PRC', 0))
    
    return max_benefit_price if max_benefit_price > 0 else (
        dscnt_sale_prc if dscnt_sale_prc > 0 else sale_prc
    )

def check_price_conditions(sale_price: int, price_conditions: list) -> bool:
    """가격 조건 체크 함수"""
    if not price_conditions:
        return True
        
    min_price = 0
    max_price = float('inf')
    
    for operator, amount in price_conditions:
        if operator in ['>=', '>']:
            min_price = max(min_price, amount if operator == '>=' else amount + 1)
        elif operator in ['<=', '<']:
            max_price = min(max_price, amount if operator == '<=' else amount - 1)
    
    return min_price <= sale_price <= max_price

def check_price_band_conditions(sale_price: int, price_band_conditions: list) -> bool:
    """가격대 조건 체크 함수"""
    if not price_band_conditions:
        return True
        
    return any(
        start_amount <= sale_price < end_amount 
        for _, start_amount, end_amount in price_band_conditions
    )

In [163]:
def check_brand_condition(metadata: dict, brands: List[str]) -> bool:
    """브랜드 조건 체크 함수"""
    if not brands:
        return True
    
    return any(
        brand in metadata.get('BRND_NM', '') or 
        brand in metadata.get('GOODS_NM', '')
        for brand in brands
    )

def check_item_condition(metadata: dict, items: List[str]) -> bool:
    """품목 조건 체크 함수"""
    if not items:
        return True
    
    return any(
        item in metadata.get('LGRP_NM', '') or 
        item in metadata.get('ARTC_NM', '') or
        item in metadata.get('SCH_KWD_NM', '')
        for item in items
    )

def check_card_discount_condition(metadata: dict, card_discounts: List[str]) -> bool:
    """카드할인 조건 체크 함수"""
    if not card_discounts:
        return True
    
    return any(
        card in metadata.get('CARD_DC_NAME', '') or 
        card in metadata.get('CARD_DC_NAME_LIST', '')
        for card in card_discounts
    )

def check_price_condition(metadata: dict, query_parts: QueryConditions) -> bool:
    """가격 조건 체크 함수"""
    if not query_parts.price_conditions and not query_parts.price_band_conditions:
        return True
    
    sale_price = get_effective_price(metadata)
    return (
        check_price_conditions(sale_price, query_parts.price_conditions) or
        check_price_band_conditions(sale_price, query_parts.price_band_conditions)
    )

def check_gdas_scr_sum(metadata: dict, gdas_condition: Optional[Tuple[float, str]]) -> bool:
    """
    평점 필터 조건이 있는 경우 통과 여부 반환. 없으면 무조건 True.
    """
    if gdas_condition is None:
        return True

    value, operator = gdas_condition
    try:
        gdas_score = float(metadata.get('GDAS_SCR_SUM', 0))
    except (ValueError, TypeError):
        return False

    if operator == ">=":
        return gdas_score >= value
    elif operator == ">":
        return gdas_score > value
    return False

In [164]:
def check_energy_grade(metadata: Dict[str, Any], energy_grade_condition: Optional[Tuple[str, str]]) -> bool:
    """에너지등급 조건 체크 함수"""
    if energy_grade_condition is None:
        return True
        
    doc_grade = metadata.get('ENERGEY_GRADE')
    if doc_grade is None or doc_grade == '' or doc_grade.strip() == '':
        return False
        
    query_grade, condition = energy_grade_condition
    
    try:
        doc_grade_num = int(doc_grade.replace('등급', ''))
        query_grade_num = int(query_grade.replace('등급', ''))
        
        if condition == 'equal':
            result = doc_grade == query_grade
            return result
        elif condition == 'above':
            result = doc_grade_num <= query_grade_num
            return result
    except ValueError as e:
        return False
    except Exception as e:
        return False
    
    return False

In [165]:
def check_mdl_lnch_dt(metadata: dict, mdl_lnch_condition) -> bool:
    """제조년월 조건 체크 함수"""
    if mdl_lnch_condition is None:
        return True

    condition = mdl_lnch_condition[0]
    mdl_lnch_dt = metadata.get('MDL_LNCH_DT')
    if not mdl_lnch_dt:
        return False

    try:
        mdl_lnch_str = str(mdl_lnch_dt)
        if len(mdl_lnch_str) != 8:
            return False
        product_year = int(mdl_lnch_str[:4])
        product_month = int(mdl_lnch_str[4:6])
        if not (1 <= product_month <= 12):
            return False
        
        product_date = product_year * 100 + product_month  # YYYYMM 형태로 변환
        
        if condition == 'exact':
            target_year, target_month = mdl_lnch_condition[1], mdl_lnch_condition[2]
            target_date = target_year * 100 + target_month
            return product_date == target_date
            
        elif condition == 'range':
            start_year, start_month, end_year, end_month = mdl_lnch_condition[1:5]
            start_date = start_year * 100 + start_month
            end_date = end_year * 100 + end_month
            return start_date <= product_date <= end_date
            
        elif condition == 'period':
            start_year, start_month, end_year, end_month = mdl_lnch_condition[1:5]
            start_date = start_year * 100 + start_month
            end_date = end_year * 100 + end_month
            return start_date <= product_date <= end_date
            
    except Exception:
        return False
    return False

In [166]:
def check_mandatory_conditions(doc: Any, query_parts: QueryConditions) -> bool:
    """필수 조건 체크 함수 - AND 조건 적용"""
    metadata = doc.metadata

    # 디버깅 필요 시 사용할 출력
    # print("\n=== Metadata 구조 확인 ===")
    # print(f"Metadata 타입: {type(metadata)}")
    # print(f"Metadata 내용: {metadata}")
    # print("=======================\n")
    
    # 모든 조건을 순차적으로 체크
    conditions = [
        check_brand_condition(metadata, query_parts.brands),
        check_item_condition(metadata, query_parts.items),
        check_card_discount_condition(metadata, query_parts.card_discounts),
        check_price_condition(metadata, query_parts),
        check_gdas_scr_sum(metadata, query_parts.gdas_scr_sum),
        check_energy_grade(metadata, query_parts.energy_grade),
        check_mdl_lnch_dt(metadata, query_parts.mdl_lnch_dt)
    ]
    
    # 모든 조건이 True여야 통과
    return all(conditions)

In [167]:
def calculate_brand_score(metadata: dict, brands: List[str]) -> float:
    """브랜드 매칭 점수 계산 함수"""
    score = 0.0
    for brand in brands:
        if brand in metadata.get('BRND_NM', ''):
            score += 5.0  # 증가: 브랜드 정확 매칭 더 중요하게
        elif brand in metadata.get('GOODS_NM', ''):
            score += 3.5  # 증가: 상품명 매칭도 중요도 상승
    return score

def calculate_item_score(metadata: dict, items: List[str]) -> float:
    """품목 매칭 점수 계산 함수"""
    score = 0.0
    for item in items:
        if item in metadata.get('ARTC_NM', ''):
            score += 4.0  # 증가: 품목정보 매칭 더 중요하게
        elif item in metadata.get('LGRP_NM', ''):
            score += 3.5  # 증가: 카테고리 매칭 중요도 상승
        elif item in metadata.get('SCH_KWD_NM', ''):
            score += 2.5  # 증가: 검색키워드 매칭도 약간 상승
    return score

def calculate_card_discount_score(metadata: dict, card_discounts: List[str]) -> float:
    """카드할인 매칭 점수 계산 함수"""
    score = 0.0
    for card in card_discounts:
        if card in metadata.get('CARD_DC_NAME', ''):
            score += 3.5  # 증가: 카드 직접 매칭
        elif card in metadata.get('CARD_DC_NAME_LIST', ''):
            score += 3.0  # 증가: 카드 리스트 매칭
    return score

def calculate_price_score(metadata: dict, query_parts: QueryConditions) -> float:
    """가격 조건 매칭 점수 계산 함수"""
    if not query_parts.price_conditions and not query_parts.price_band_conditions:
        return 0.0
    
    sale_price = get_effective_price(metadata)
    if (check_price_conditions(sale_price, query_parts.price_conditions) or
        check_price_band_conditions(sale_price, query_parts.price_band_conditions)):
        return 3.0  # 증가: 가격 조건 만족 시 더 높은 점수
    
    return 0.0

def calculate_gdas_scr_sum_score(conditions: QueryConditions, product: tuple) -> float:
    """
    조건 필터 통과 여부와 별개로, 스코어 계산을 위한 별점 기반 점수 부여 --> 별점순으로 정렬하기 위함
    만약 정렬이 필요 없다면,
    def score_document() 에서
    calculate_gdas_scr_sum_score(metadata, query_parts.gdas_scr_sum)
    를 주석처리
    """
    if conditions.get("GDAS_SCR_SUM") is None:
        return 0.0

    try:
        raw = conditions.get('GDAS_SCR_SUM')
        product_score = float(raw) if raw else 0.0
    except (IndexError, ValueError, TypeError):
        product_score = 0.0

    return product_score / 5.0  # 최대 5점 만점 기준 정규화


In [168]:
def calculate_energy_grade_score(metadata: Dict[str, Any], energy_grade_condition: Optional[Tuple[str, str]]) -> int:
    """
    에너지등급 매칭에 대한 점수 계산
    int형태로 계산되며, 등급 차이당 1씩 감점, 최대 5점
    만약 정렬이 필요 없다면,
    def score_document() 에서
    calculate_gdas_scr_sum_score(metadata, query_parts.gdas_scr_sum)
    를 주석처리
    """
    if energy_grade_condition is None:
        return 0
        
    doc_grade = metadata.get('energy_grade')
    if doc_grade is None:
        return 0
        
    query_grade, condition = energy_grade_condition
    
    doc_grade_num = int(doc_grade.replace('등급', ''))
    query_grade_num = int(query_grade.replace('등급', ''))
    
    if condition == 'equal':
        return 5 if doc_grade == query_grade else 0
    else:
        grade_diff = query_grade_num - doc_grade_num
        return max(0, 5 - grade_diff)  # 등급 차이당 1씩 감점, 최대 5점

In [169]:
def calculate_mdl_lnch_dt_score(metadata: dict, mdl_lnch_condition) -> float:
    """
    제조년월 점수 계산 (명확한 날짜: pass/fail, 범위: 현재와의 차이에 따른 점수)
    만약 정렬이 필요 없다면,
    def score_document() 에서
    calculate_mdl_lnch_dt_score(metadata, query_parts.mdl_lnch_dt)
    를 주석처리
    """
    if mdl_lnch_condition is None:
        return 0.0

    condition = mdl_lnch_condition[0]
    mdl_lnch_dt = metadata.get('MDL_LNCH_DT')
    if not mdl_lnch_dt:
        return 0.0

    try:
        mdl_lnch_str = str(mdl_lnch_dt)
        if len(mdl_lnch_str) != 8:
            return 0.0
        product_year = int(mdl_lnch_str[:4])
        product_month = int(mdl_lnch_str[4:6])
        if not (1 <= product_month <= 12):
            return 0.0
        
        # 현재 날짜 기준으로 월 차이 계산
        from datetime import datetime
        current_date = datetime.now()
        current_year = current_date.year
        current_month = current_date.month
        
        # 제품 날짜와 현재 날짜의 월 차이 계산
        def calculate_month_diff(year1, month1, year2, month2):
            return (year2 - year1) * 12 + (month2 - month1)
        
        month_diff_from_current = calculate_month_diff(product_year, product_month, current_year, current_month)
        
        if condition == 'exact':
            # 명확한 날짜 조건: pass/fail만
            target_year, target_month = mdl_lnch_condition[1], mdl_lnch_condition[2]
            target_date = target_year * 100 + target_month
            product_date = product_year * 100 + product_month
            
            if product_date == target_date:
                return 5.0  # 정확히 일치하면 5점
            else:
                return 0.0  # 일치하지 않으면 0점
                
        elif condition in ['range', 'period']:
            # 범위 날짜 조건: 현재와의 차이에 따른 점수
            start_year, start_month, end_year, end_month = mdl_lnch_condition[1:5]
            start_date = start_year * 100 + start_month
            end_date = end_year * 100 + end_month
            product_date = product_year * 100 + product_month
            
            if start_date <= product_date <= end_date:
                # 범위 내에 있으면 기본 5점에서 시작
                base_score = 5.0
                
                # 현재와의 월 차이에 따른 감점 (1개월 차이당 -0.1점)
                penalty = abs(month_diff_from_current) * 0.1
                
                return max(0.0, base_score - penalty)
            else:
                # 범위 밖이면 0점
                return 0.0
                
    except Exception:
        return 0.0
    
    return 0.0

In [170]:
def calculate_gdas_cnt(metadata: dict, sort_direction: Optional[str]) -> float:
    """리뷰개수 점수 계산 함수"""
    if sort_direction is None:
        return 0.0
    
    try:
        gdas_cnt = metadata.get('GDAS_CNT')
        if gdas_cnt is None:
            return 0.0
        
        gdas_cnt_value = int(gdas_cnt)
        
        if sort_direction == 'desc':
            return float(gdas_cnt_value) * 1000
        elif sort_direction == 'asc':
            # 적은 순: 0이면 1점, 1이면 2점 방식으로 역순 점수
            return 1.0 + float(gdas_cnt_value) * 1000
        
    except (ValueError, TypeError):
        return 0.0
    
    return 0.0

In [171]:
def calculate_sale_qty(metadata: dict, sort_direction: Optional[str]) -> float:
    """판매량(한달) 점수 계산 함수"""
    if sort_direction is None:
        return 0.0
    
    try:
        sale_qty = metadata.get('SALE_QTY')
        if sale_qty is None:
            return 0.0
        
        sale_qty_value = int(sale_qty)
        
        if sort_direction == 'desc':
            return float(sale_qty_value)
        elif sort_direction == 'asc':
            # 적은 순: 0이면 1점, 1이면 2점 방식으로 역순 점수
            return 1.0 + float(sale_qty_value)
        
    except (ValueError, TypeError):
        return 0.0
    
    return 0.0

In [172]:
def calculate_sales_unit(metadata: dict, sort_direction: Optional[str]) -> float:
    """단위판매량 점수 계산 함수"""
    if sort_direction is None:
        return 0.0
    
    try:
        sales_unit = metadata.get('SALES_UNIT')
        if sales_unit is None:
            return 0.0
        
        sales_unit_value = int(sales_unit)
        
        if sort_direction == 'desc':
            return float(sales_unit_value)
        elif sort_direction == 'asc':
            # 적은 순: 0이면 1점, 1이면 2점 방식으로 역순 점수
            return 1.0 + float(sales_unit_value)
        
    except (ValueError, TypeError):
        return 0.0
    
    return 0.0

In [173]:
def score_document(doc: Any, query_parts: QueryConditions) -> float:
    """문서 스코어링 함수 - 가중치 적용"""
    metadata = doc.metadata
    
    # 각 조건별 점수 계산
    scores = [
        calculate_brand_score(metadata, query_parts.brands),
        calculate_item_score(metadata, query_parts.items),
        calculate_card_discount_score(metadata, query_parts.card_discounts),
        calculate_price_score(metadata, query_parts),
        # calculate_gdas_scr_sum_score(metadata, query_parts.gdas_scr_sum),
        # calculate_energy_grade_score(metadata, query_parts.energy_grade),
        # calculate_mdl_lnch_dt_score(metadata, query_parts.mdl_lnch_dt),

        # 정렬의 경우 정렬 효과를 확실하게 하기 위해 * 1000 을 곱함
        calculate_gdas_cnt(metadata, query_parts.sort_gdas_cnt) * 1000,
        calculate_sale_qty(metadata, query_parts.sort_sale_qty) * 1000,
        calculate_sales_unit(metadata, query_parts.sort_sales_unit) * 1000
    ]
    
    # 총점 반환
    return sum(scores)

In [174]:
def process_batch(batch_data: Tuple[List[Any], List[float]], query_parts: QueryConditions, batch_id: int) -> Tuple[List[Dict[str, Any]], int]:
    """배치 단위 처리 함수"""
    docs, scores = batch_data
    results = []
    
    for doc, vector_score in zip(docs, scores):
        if check_mandatory_conditions(doc, query_parts):
            relevance_score = score_document(doc, query_parts)
            results.append({
                'document': doc,
                'vector_score': vector_score,
                'relevance_score': relevance_score,
                'combined_score': vector_score * 0.3 + relevance_score * 0.7  # 수정: 규칙 기반 점수의 비중을 높임
            })
    
    return results, batch_id

In [175]:
def hybrid_search(query: str, k: int = 5, batch_size: int = 1000, max_concurrent_batches: int = 5) -> List[Dict[str, Any]]:
    """하이브리드 검색 실행 함수 - 대용량 데이터 처리 최적화"""
    start_time = time.time()
    print(f"🔍 검색 시작: {query}")
    
    query_parts = analyze_query(query)
    
    initial_search_size = min(100000, batch_size * 100)
    print(f"📊 초기 검색 크기: {initial_search_size:,}개")
    
    vector_search_start = time.time()
    faiss_results = vectorstore.similarity_search_with_score(query, k=initial_search_size)
    print(f"⚡ 벡터 검색 소요 시간: {time.time() - vector_search_start:.2f}초")
    
    docs = [doc for doc, _ in faiss_results]
    scores = [score for _, score in faiss_results]
    total_batches = (len(docs) + batch_size - 1) // batch_size
    batches = [
        (
            docs[i:i + batch_size],
            scores[i:i + batch_size]
        )
        for i in range(0, len(docs), batch_size)
    ]
    
    print(f"📦 총 {total_batches}개 배치 처리 시작")
    
    all_results = []
    processed_batches = 0
    
    with ThreadPoolExecutor(max_workers=max_concurrent_batches) as executor:
        future_to_batch = {
            executor.submit(process_batch, batch, query_parts, i): i
            for i, batch in enumerate(batches)
        }
        
        for future in as_completed(future_to_batch):
            batch_results, batch_id = future.result()
            all_results.extend(batch_results)
            processed_batches += 1
            
            if processed_batches % 10 == 0:
                print(f"⏳ 진행률: {processed_batches}/{total_batches} 배치 처리 완료")
    
    sorted_results = sorted(all_results, key=lambda x: x['combined_score'], reverse=True)[:k]
    
    print(f"✅ 검색 완료: 총 {len(all_results):,}개 결과 중 상위 {k}개 선택")
    print(f"⌛ 총 소요 시간: {time.time() - start_time:.2f}초")
    
    return sorted_results

In [176]:
def rerank_with_llm(query: str, docs: List[Tuple[Any, float]], max_docs_for_rerank: int = None) -> List[Tuple[Any, float]]:
    """
    LLM을 사용한 재순위화 함수
    
    Args:
        query: 검색 쿼리
        docs: (document, score) 튜플의 리스트
        max_docs_for_rerank: 재순위화할 최대 문서 수 (None이면 자동 계산)
    """
    start_time = time.time()
    print(f"\n🔄 재순위화 시작: {query}")
    
    if not docs:
        print("⚠️ 재순위화할 문서가 없습니다.")
        return []
    
    # 문서 크기 계산 및 제한 설정
    def estimate_tokens(doc: Any) -> int:
        """문서의 대략적인 토큰 수 추정"""
        content = doc.page_content
        metadata_str = str(doc.metadata)
        return (len(content) + len(metadata_str)) // 4

    # GPT-4의 컨텍스트 제한을 고려한 문서 수 계산
    MAX_TOKENS = 8000  # GPT-4 컨텍스트의 안전한 제한값
    TOKENS_FOR_PROMPT = 500  # 프롬프트 자체에 필요한 토큰 예약
    available_tokens = MAX_TOKENS - TOKENS_FOR_PROMPT
    
    # 문서별 토큰 수 계산
    docs_with_tokens = [(doc, score, estimate_tokens(doc)) 
                       for doc, score in docs]
    
    # 동적으로 처리 가능한 문서 수 계산
    total_tokens = 0
    docs_to_process = 0
    for _, _, tokens in docs_with_tokens:
        if total_tokens + tokens > available_tokens:
            break
        total_tokens += tokens
        docs_to_process += 1
    
    # 최소/최대 문서 수 제한 적용
    MIN_DOCS = 5
    MAX_DOCS = 30
    docs_to_process = max(MIN_DOCS, min(docs_to_process, MAX_DOCS))
    
    # 사용자가 지정한 제한이 있다면 적용
    if max_docs_for_rerank is not None:
        docs_to_process = min(docs_to_process, max_docs_for_rerank)
    
    print(f"📊 문서 처리 계획:")
    print(f"- 총 입력 문서: {len(docs)}개")
    print(f"- 처리 가능 문서: {docs_to_process}개")
    print(f"- 예상 토큰 사용량: {total_tokens:,} / {MAX_TOKENS:,}")
    
    # 처리할 문서 선택
    docs_to_rerank = docs[:docs_to_process]
    total_docs = len(docs_to_rerank)
    
    try:
        llm_start = time.time()
        llm = ChatOpenAI(model_name="gpt-4", temperature=0)
        
        # 프롬프트 템플릿
        prompt = PromptTemplate.from_template("""
사용자 쿼리: {query}

당신의 작업:
1. 아래 {total_docs}개의 문서를 관련도 기준으로 정렬하세요.
2. 반드시 모든 문서를 포함해야 합니다.
3. 숫자만 쉼표로 구분하여 응답하세요.

규칙:
- 정확히 {total_docs}개의 번호를 포함해야 합니다.
- 1부터 {total_docs}까지의 번호만 사용하세요.
- 각 번호는 한 번만 사용해야 합니다.
- 다른 설명 없이 숫자와 쉼표만 사용하세요.

문서들:
{docs}

응답 형식: 1,4,2,3 (숫자와 쉼표만 사용)
""")
        
        # 문서 포맷팅
        combined = "\n---\n".join(
            f"문서 {i+1}:\n제목: {doc.metadata.get('GOODS_NM', '정보 없음')}\n내용: {doc.page_content}"
            for i, (doc, _) in enumerate(docs_to_rerank)
        )
        
        # LLM 호출
        chain = prompt | llm | StrOutputParser()
        response = chain.invoke({"query": query, "docs": combined, "total_docs": total_docs})
        print(f"⚡ LLM 처리 소요 시간: {time.time() - llm_start:.2f}초")
        
        # 응답 파싱 및 검증
        parsing_start = time.time()
        try:
            indices = [int(i.strip()) - 1 for i in response.strip().split(",") if i.strip().isdigit()]
            
            # 검증
            if len(indices) != total_docs:
                print(f"⚠️ 경고: LLM이 {total_docs}개가 아닌 {len(indices)}개의 문서를 반환했습니다.")
                # 누락된 인덱스 찾기
                used_indices = set(indices)
                all_indices = set(range(total_docs))
                missing_indices = all_indices - used_indices
                
                # 누락된 인덱스 추가
                indices.extend(sorted(missing_indices))
                print(f"🔄 누락된 문서 {len(missing_indices)}개를 마지막에 추가했습니다.")
            
            # 중복 체크
            if len(set(indices)) != len(indices):
                print("⚠️ 경고: 중복된 인덱스가 발견되었습니다.")
                indices = list(dict.fromkeys(indices))  # 중복 제거
                # 누락된 인덱스 추가
                all_indices = set(range(total_docs))
                missing_indices = all_indices - set(indices)
                indices.extend(sorted(missing_indices))
            
            # 범위 검증
            indices = [i for i in indices if 0 <= i < total_docs]
            
        except Exception as parse_error:
            print(f"⚠️ 응답 파싱 오류: {parse_error}")
            indices = list(range(total_docs))
        
        # 결과 로깅
        print("\n🔍 재순위화 결과:")
        for rank, idx in enumerate(indices, 1):
            doc = docs_to_rerank[idx][0]
            goods_no = doc.metadata.get('GOODS_NO', '정보 없음')
            goods_nm = doc.metadata.get('GOODS_NM', '정보 없음')
            print(f"순위 {rank}. 상품번호: {goods_no} - {goods_nm}")
        
        # 재순위화된 문서 생성
        reranked = [docs_to_rerank[i] for i in indices]
        
        # 나머지 문서 추가
        if len(docs) > docs_to_process:
            remaining_docs = docs[docs_to_process:]
            print(f"📎 나머지 {len(remaining_docs):,}개 문서 추가")
            reranked.extend(remaining_docs)
        
        print(f"⚡ 결과 파싱 소요 시간: {time.time() - parsing_start:.2f}초")
        print(f"✅ 재순위화 완료: 총 {len(reranked):,}개 결과")
        print(f"⌛ 총 소요 시간: {time.time() - start_time:.2f}초")
        
        return reranked
        
    except Exception as e:
        print(f"❌ 재순위화 중 오류 발생: {e}")
        print(f"⌛ 오류까지 소요 시간: {time.time() - start_time:.2f}초")
        return docs  # 오류 발생 시 원본 순서 유지

In [177]:
def print_search_results(results: List[Dict[str, Any]], title: str = "검색 결과"):
    """검색 결과 출력 함수"""
    print(f"\n📦 {title}")
    for i, result in enumerate(results, 1):
        print(f"[결과 {i}] 최종점수: {result['combined_score']:.4f}")
        doc = result['document']
        print(f"{doc.page_content}")
        try:
            print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
            print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
            print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
            print(f"- 품목정보: {doc.metadata.get('ARTC_NM', '정보 없음')}")
            print(f"- 카테고리: {doc.metadata.get('LGRP_NM', '정보 없음')}")
            print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
            print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
            print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
            print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
            print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
            print(f"- 평점: {doc.metadata.get('GDAS_SCR_SUM', '정보 없음')}")
            print(f"- 에너지등급: {doc.metadata.get('ENERGEY_GRADE', '정보 없음')}")
            print(f"- 제조년월: {doc.metadata.get('MDL_LNCH_DT', '정보 없음')}")
            print(f"- 리뷰개수: {doc.metadata.get('GDAS_CNT', '정보 없음')}")
            print(f"- 판매량(한달): {doc.metadata.get('SALE_QTY', '정보 없음')}")
            print(f"- 단위판매량: {doc.metadata.get('SALES_UNIT', '정보 없음')}")
            print(f"- 주요 특징 및 기능:")
            feature_values = doc.metadata.get('OPT_VAL_DESC', '').split(',')
            feature_titles = doc.metadata.get('OPT_DISP_NM', '').split(',')
            for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                print(f"  - {title} : {value}")
                if i >= 2: break
            print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
        except Exception as e:
            print(f"메타데이터 출력 중 오류: {str(e)}")
        print("-" * 60)

In [178]:
# 검색 실행
# query = "세탁기 별점 3.1점 이상"
query = "현대카드 할인하는 LG전자 TV 중 단위판매량 많은 순"
# query = "세탁기 3등급 이상"
# query = "세탁기 2024년 3월 생산"
# query = "세탁기 3개월 이내 생산"

# 1. 하이브리드 검색 실행
hybrid_results = hybrid_search(
    query=query,
    k=5,
    batch_size=1000,  # 배치 크기
    max_concurrent_batches=5  # 동시 처리할 최대 배치 수
)

print_search_results(hybrid_results, "하이브리드 검색 결과")

🔍 검색 시작: 현대카드 할인하는 LG전자 TV 중 단위판매량 많은 순

✅ 브랜드 :: ['LG전자']

✅ 아이템 :: ['TV']

✅ 카드할인 :: ['현대카드']

✅ 가격 조건 :: []

✅ 가격대 :: []

✅ 평점 :: None

✅ 에너지등급 :: None

✅ 제조년월 :: None

✅ 리뷰개수 정렬 :: None

✅ 판매량 정렬 :: desc

✅ 단위판매량 정렬 :: desc
📊 초기 검색 크기: 100,000개
⚡ 벡터 검색 소요 시간: 1.50초
📦 총 100개 배치 처리 시작
⏳ 진행률: 10/100 배치 처리 완료
⏳ 진행률: 20/100 배치 처리 완료
⏳ 진행률: 30/100 배치 처리 완료
⏳ 진행률: 40/100 배치 처리 완료
⏳ 진행률: 50/100 배치 처리 완료
⏳ 진행률: 60/100 배치 처리 완료
⏳ 진행률: 70/100 배치 처리 완료
⏳ 진행률: 80/100 배치 처리 완료
⏳ 진행률: 90/100 배치 처리 완료
⏳ 진행률: 100/100 배치 처리 완료
✅ 검색 완료: 총 110개 결과 중 상위 5개 선택
⌛ 총 소요 시간: 1.91초

📦 하이브리드 검색 결과
[결과 1] 최종점수: 1193509.3136
LG전자 브랜드의 이동식TV 제품이며, 상품명 68cm LG 스탠바이미 TV 27ART10DSPL . 모델명: 27ART10DSPL.AKR. 제품의 주요 특징은 크기: 27인치(68cm), 종류: LED, 해상도: Full-HD, 기본 주사율: 60Hz, 엔진: 알파7 Gen4, 부가기능: 넷플릭스, 유튜브, 인터넷, USB재생, 블루투스, 출시년도: 2024년, 화면 타입: 평면형, 스피커 출력: 10W, 음향효과: 돌비애트모스, 지원단자: HDMI, 소비전력: 76W, 색상: 베이지계열, HDMI 포트수: 1포트, USB 포트수: 1포트, TV 설치 유형: 스탠드형 . 카테고리: TV·영상가전. 가격 정보: 판매가 1,000,000원, 현대카드 카드 사용 시 10% 할인되어 최대 900,00

In [179]:
# 2. LLM 재순위화
reranked_results = rerank_with_llm(
    query=query,
    docs=[(result['document'], result['combined_score']) for result in hybrid_results],
    max_docs_for_rerank=20
)

llm_results = [
    {
        'document': doc,
        'combined_score': score,
        'vector_score': 0,
        'relevance_score': score
    }
    for doc, score in reranked_results
]

print_search_results(llm_results, "하이브리드 + LLM 재순위화 결과")


🔄 재순위화 시작: 현대카드 할인하는 LG전자 TV 중 단위판매량 많은 순
📊 문서 처리 계획:
- 총 입력 문서: 5개
- 처리 가능 문서: 5개
- 예상 토큰 사용량: 1,816 / 8,000
⚡ LLM 처리 소요 시간: 1.20초

🔍 재순위화 결과:
순위 1. 상품번호: 0038179506 - 68cm LG 스탠바이미 TV 27ART10DSPL (돌비O, 후면 카밍베이지)
순위 2. 상품번호: 0040192198 - 25년 신모델 68cm LG전자 스탠바이미2 27LX6TKGA
순위 3. 상품번호: 0022381958 - LG 울트라기어 (VA) 24GS50F (1920 x 1080 FHD)
순위 4. 상품번호: 0023335625 - 24MS500.BKR PC 일반 모니터 FHD 60.4cm
순위 5. 상품번호: 0021404358 - 189cm LG QNED TV 75QNED85TKA (스탠드형)
⚡ 결과 파싱 소요 시간: 0.00초
✅ 재순위화 완료: 총 5개 결과
⌛ 총 소요 시간: 1.21초

📦 하이브리드 + LLM 재순위화 결과
[결과 1] 최종점수: 1193509.3136
LG전자 브랜드의 이동식TV 제품이며, 상품명 68cm LG 스탠바이미 TV 27ART10DSPL . 모델명: 27ART10DSPL.AKR. 제품의 주요 특징은 크기: 27인치(68cm), 종류: LED, 해상도: Full-HD, 기본 주사율: 60Hz, 엔진: 알파7 Gen4, 부가기능: 넷플릭스, 유튜브, 인터넷, USB재생, 블루투스, 출시년도: 2024년, 화면 타입: 평면형, 스피커 출력: 10W, 음향효과: 돌비애트모스, 지원단자: HDMI, 소비전력: 76W, 색상: 베이지계열, HDMI 포트수: 1포트, USB 포트수: 1포트, TV 설치 유형: 스탠드형 . 카테고리: TV·영상가전. 가격 정보: 판매가 1,000,000원, 현대카드 카드 사용 시 10% 할인되어 최대 900,000원. 해시태그: #27ART10DSPL#27인치TV#소형TV#이동형TV#무

In [180]:
def get_product_info_json(reranked_results):
    """
    reranked_results에서 상품 정보를 JSON 형태로 반환하는 함수

    Args:
        reranked_results: (document, score) 튜플의 리스트

    Returns:
        List[Dict[str, Any]]: 상품 정보가 담긴 JSON 형태의 리스트
    """
    product_info_list = []

    for doc, score in reranked_results:
        metadata = doc.metadata
        product_info = {
            'goods_no': metadata.get('GOODS_NO', None),
            'goods_nm': metadata.get('GOODS_NM', None),
            'goods_stat': metadata.get('GOODS_STAT_SCT_NM', None),
            'brand_nm': metadata.get('BRND_NM', None),
            'artc_nm': metadata.get('ARTC_NM', None),
            'lgrp_nm': metadata.get('LGRP_NM', None),
            'sale_prc': int(metadata.get('SALE_PRC', 0)),
            'dscnt_sale_prc': int(metadata.get('DSCNT_SALE_PRC', 0)),
            'max_benefit_price': int(metadata.get('MAX_BENEFIT_PRICE', 0)),
            'card_dc_rate': metadata.get('CARD_DC_RATE', '0'),
            'card_dc_name_list': metadata.get('CARD_DC_NAME_LIST', None),
            'gdas_scr_sum': metadata.get('GDAS_SCR_SUM', None),
            'features': []
        }

        # 주요 특징 및 기능 추가
        feature_values = metadata.get('OPT_VAL_DESC', '').split(',')
        feature_titles = metadata.get('OPT_DISP_NM', '').split(',')
        for title, value in zip(feature_titles, feature_values):
            if title and value:
                product_info['features'].append({
                    'title': title,
                    'value': value
                })

        product_info_list.append(product_info)

    return product_info_list

In [181]:
get_product_info_json([(item['document'], item['combined_score']) for item in llm_results])

[{'goods_no': '0038179506',
  'goods_nm': '68cm LG 스탠바이미 TV 27ART10DSPL (돌비O, 후면 카밍베이지)',
  'goods_stat': '정상상품',
  'brand_nm': 'LG전자',
  'artc_nm': '이동식TV',
  'lgrp_nm': 'TV·영상가전',
  'sale_prc': 1000000,
  'dscnt_sale_prc': 1000000,
  'max_benefit_price': 900000,
  'card_dc_rate': 10,
  'card_dc_name_list': '현대카드,삼성카드,롯데카드,네이버페이,신한카드,KB국민카드',
  'gdas_scr_sum': 0.0,
  'features': [{'title': '크기', 'value': '27인치(68cm)'},
   {'title': '종류', 'value': 'LED'},
   {'title': '해상도', 'value': 'Full-HD'},
   {'title': '기본 주사율', 'value': '60Hz'},
   {'title': '엔진', 'value': '알파7 Gen4'},
   {'title': '스마트기능', 'value': '넷플릭스'},
   {'title': '스마트기능', 'value': '유튜브'},
   {'title': '스마트기능', 'value': '인터넷'},
   {'title': '부가기능', 'value': 'USB재생'},
   {'title': '부가기능', 'value': '블루투스'},
   {'title': '출시년도', 'value': '2024년'},
   {'title': '화면 타입', 'value': '평면형'},
   {'title': '스피커 출력', 'value': '10W'},
   {'title': '음향효과', 'value': '돌비애트모스'},
   {'title': '지원단자', 'value': 'HDMI'},
   {'title': '소비전력', 